In [10]:
import csv
import json
import sys
import pandas as pd
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson
pd.set_option("display.max_rows", None, "display.max_columns", None)

user_cookie = dict()

for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

urllist = ["https://lpl.qq.com/es/stats.shtml?bmid=7325",
        "https://lpl.qq.com/es/stats.shtml?bmid=6909"]

test_url = "https://lpl.qq.com/es/stats.shtml?bmid=6909"

#This is pulling the champion data.
champ_dd = requests.get("https://ddragon.leagueoflegends.com/cdn/11.8.1/data/en_US/champion.json", cookies=user_cookie)
champ_json = champ_dd.json()
champ_name = champ_json["data"] #this creates a list of all the champions
champ_key_dict = {}
for i in champ_name:
    champ_key_dict.update({champ_json["data"][i]["key"]:i})
#pprint.pprint(champ_dd.json())
#control = {'side':'left', 'count':0, 'records':0}
#json_data['bMatchId']
#jd = json.loads(json_data["sMatchInfo"][0]['battleInfo']['BattleData'])
#jd['left']['players']

def lpl_url_for_request_scraping(raw_url, test):
    base_url = "https://lpl.qq.com/web201612/data/LOL_MATCH_DETAIL_"
    js = ".js"
    match_num = raw_url[-4:]
    new_url = base_url + match_num + js
    if test == True:
        pprint.pprint(new_url)
    return (new_url)

#Get the lpl json data. 
def lpl_get_match_data(url, test):
    good_url = lpl_url_for_request_scraping(url, False)
    json_file = requests.get(good_url)
    json_content = json.loads(json_file.text[12:-1]) # or demjson.decode(json_file.content[12:-1])
    #assert json_file.status_code == 200
    if test == True:
        pprint.pprint(json_content)
    return(json_content)

def lpl_build_dataframe(input_match_data, test):
    jd = json.loads(input_match_data["sMatchInfo"][0]['battleInfo']['BattleData'])
    #'bMatchId' is what matches the URL
    side = 'left'
    count = 0
    records = 0
    while records < 10:
        if count == 5:
            count = 0
            side = 'right'
        else:
            player_info = jd[side]['players'][count]
            champion_name = champ_key_dict[str(player_info['hero'])]
            gameId = json.loads(input_match_data['bMatchId'])
            player_info.update({"hero": champion_name , "gameId": gameId})
            if count == 0 and records == 0:
                df = pd.DataFrame([player_info])
                records = records + 1
                count = count + 1
            else:
                new_df = pd.DataFrame([player_info])
                count = count + 1
                records = records + 1
                df = df.append(new_df) 
    return (df)


y = lpl_get_match_data(test_url, False)
z = lpl_build_dataframe(y, False)

display(z)

,kill,death,assist,gold,lasthit,hero,name,memberId,skill-1,skill-2,equip,win,totalDamage,totalDamageToChamp,pDamageToChamp,mDamageDealtToChamp,pDamageDealt,mDamageDealt,largestCriticalStrike,totalHeal,totalDamageTaken,pDamageTaken,mDamageTaken,neutralKilled,neutralKilledTJungle,neutralKilledEJungle,wardsPlaced,wardsKilled,visionWardsBought,dKills,tKills,qKills,pKills,towerKills,inhibitorKills,deadTime,level,killingSprees,largestMultiKill,largestKillingSpree,firstBlood,firstTower,gameId
0,0,0,2,10711,276,Karma,TES369,2147,4,12,"{'game-equip-1': '6655', 'game-equip-2': '2055...",0,152963,18444,1151,17292,26467,122465,0,4249,11374,5898,5332,4,4,0,13,10,3,0,0,0,0,0,0,0,16,0,0,0,,,6909
0,1,2,1,10106,197,Pantheon,TESKarsa,53,11,4,"{'game-equip-1': '6630', 'game-equip-2': '1028...",0,195713,8585,7367,987,166926,10791,0,14062,29694,20572,8904,174,115,3,12,12,12,0,0,0,0,0,0,45,14,0,1,0,,,6909
0,2,1,0,13284,343,Syndra,TESknight,2562,4,12,"{'game-equip-1': '6655', 'game-equip-2': '3165...",0,213609,13105,224,12881,16454,194143,0,1860,8972,3300,5609,40,24,10,5,13,1,0,0,0,0,1,0,54,16,1,1,2,1,,6909
0,0,2,2,13742,360,Samira,TESJackeyLove,1832,4,7,"{'game-equip-1': '1018', 'game-equip-2': '3033...",0,207282,20740,18828,822,188414,16933,468,7155,24805,12647,11845,25,19,0,10,18,5,0,0,0,0,2,0,89,16,0,0,0,,1,6909
0,0,4,1,6774,54,Leona,TESZhuo,2295,4,14,"{'game-equip-1': '3068', 'game-equip-2': '3857...",0,13957,2441,671,1267,3495,2701,0,1093,12588,4418,8043,0,0,0,61,30,19,0,0,0,0,0,0,97,10,0,0,0,,,6909
0,1,0,6,13468,316,Aatrox,SNBin,2687,4,12,"{'game-equip-1': '3053', 'game-equip-2': '6630...",1,203155,7861,7861,0,198925,0,0,17397,34597,19408,14720,36,18,4,18,4,7,0,0,0,0,1,0,0,18,0,1,0,,,6909
0,1,1,6,10867,205,Nidalee,SNSofM,592,4,11,"{'game-equip-1': '3157', 'game-equip-2': '3111...",1,172308,10724,533,9974,20370,139802,0,23808,28808,18012,10384,161,122,8,60,27,13,0,0,0,0,2,0,21,15,0,1,0,,,6909
0,2,0,5,12602,299,Viktor,SNAngel,2268,4,12,"{'game-equip-1': '2420', 'game-equip-2': '3916...",1,183858,17497,24,17472,17503,165789,0,1826,9827,3699,5860,20,18,0,15,8,5,0,0,0,0,0,1,0,16,1,1,2,,,6909
0,5,0,3,16176,359,Kaisa,SNhuanfeng,2284,1,4,"{'game-equip-1': '6671', 'game-equip-2': '6676...",1,240765,18459,11886,5925,213184,24562,545,6272,15707,9830,5696,51,41,0,13,12,5,1,0,0,0,3,0,0,17,1,2,5,,,6909
0,0,2,8,7407,36,Galio,SNON,3509,4,3,"{'game-equip-1': '3190', 'game-equip-2': '3860...",1,22792,3865,56,3808,954,14561,0,1651,11526,9638,1394,0,0,0,55,10,16,0,0,0,0,0,0,68,12,0,0,0,,,6909


Using legacy setup.py install for demjson, since package 'wheel' is not installed.
    Running setup.py install for demjson: started
    Running setup.py install for demjson: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
def get_data_lpl(json_content, test):
    list_list = []
    jd = json.loads(json_content["sMatchInfo"][0]['battleInfo']['BattleData']) #this returns a json
    keys = jd.keys()
    control = {'side':'left', 'count':0, 'records':0}
    gameId= json_content['bMatchId']
    #this subsection is to move between the 'right' and 'left' team lists because that's how this info is nested.
    while control['records'] < 10: #set this to 1 to get it to run for only one player
        output_list = []
        if control['count'] == 5:
            control['side'] = 'right'
            control['count'] = 0
        else: #I think the champion id -> champion modifier goes in here.
            side = control['side']
            count = control['count']
            data = jd[side]['players'][count] #dicts
        for i in a_keys:
            try:
                if i == 'hero': #this statement worked
                    output_list.append(str(champ_key_dict[jd[side]['players'][count][i]]))
                elif data[i]:
                    #print('yes')
                    output_list.append(data[i]) #list of all of the player info, which is currently fucking up
                elif i == 'firstBlood': #have to have this statement in here because apparently if the player didn't FB they just leave the line empty and python picks it up as False
                        if not data[i]:
                            data[i] = '0'
                            output_list.append(data[i])
                        else:
                            data[i] = '1'
                            output_list.append(data[i])
                elif i == 'game-id':
                    output_list.append(gameId)
                elif i == 'side':
                    output_list.append(control['side'])
                else:
                    print (i, 'not in lpl headers')
            except Exception as e: # probably dont need this full line because I'm doing something with the exception
                if i in b_keys:
                    print (i, "in non-lpl headers")
                    pass
                elif i == 'game-id' or 'side':
                    continue
                else:
                    print ("exception", i)
        list_list.append(output_list)
        control['count'] = control['count'] + 1
        control['records'] = control['records'] + 1
        print('done')
    if test == True:
        pprint.pprint(list_list)
    return(list_list)